In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

from keras.layers import Dense, Input, Embedding, Reshape, Dropout, LeakyReLU, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras import backend as Backend
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
Backend.tensorflow_backend._get_available_gpus()
import tensorflow as k
config = k.ConfigProto()
config.gpu_options.allow_growth = True

from pymongo import MongoClient
from bson.objectid import ObjectId

Using TensorFlow backend.


In [2]:
def load_Data(url):
    client = MongoClient(url)
    db = client['lounjee']
    matches = db['matches']
    users = db['users']
    feedbacks = db['meetingFeedbackAnswers']
    dialogs = db['dialogs']
    
    result = []
    for item in users.find():
        result.append(str(item['_id']))
    df = pd.DataFrame(data=np.zeros((len(result),len(result))), index=result,dtype=np.int8, columns=result)
    
    def find_item(df,usera,userb,value=0):
        df[userb][usera] = value
        return df[userb][usera]
        
    for item in matches.find():
        try:
            if item['stateA']['type'] == 'accepted':
                find_item(df,str(item['userA']),str(item['userB']),1)
            elif item['stateA']['type'] == 'accepting':
                find_item(df,str(item['userA']),str(item['userB']),1)
            elif item['stateA']['type'] == 'postponed':
                find_item(df,str(item['userA']),str(item['userB']),1)
    #         elif item['stateA']['type'] == 'reporting':
    #             find_item(df,str(item['userA']),str(item['userB']),-5)
        except Exception:
            pass
    
    for item in users.find():
        try:
            if item['favorites']:
                for f in item['favorites']:
                    find_item(df,str(item['_id']),str(f['_id']),1)
        except Exception:
            pass
    
    df = df[(df.T != 0).any()]
    
    result=[]
    for i in df.index.values:
        for idx,val in enumerate(df.loc[i]):
            result.append({'usera':i,'userb':(df.loc[i].index)[idx],'rating':val})

    return pd.DataFrame(result)

In [3]:
def data_Engineering(ratings,data, sampling=1):
    # Random under sampling
    if sampling is 1:
        count_class_0, count_class_1 = ratings.rating.value_counts()
        df_class_0 = ratings[ratings['rating'] == 0]
        df_class_1 = ratings[ratings['rating'] == 1]
        df_class_0_under = df_class_0.sample(count_class_1)
        ratings = pd.concat([df_class_0_under, df_class_1], axis=0)
    elif sampling is 2:
        count_class_0, count_class_1 = ratings.rating.value_counts()
        df_class_0 = ratings[ratings['rating'] == 0]
        df_class_1 = ratings[ratings['rating'] == 1]
        df_class_1_over = df_class_1.sample(count_class_0, replace=True)
        ratings = pd.concat([df_class_0, df_class_1_over], axis=0)
    
    ## reset index
    ratings.reset_index(inplace=True,drop=True)
    ratings.head()
    
    ## One hot encoding
    def dum(df,name):
        dummies = df[name].str.get_dummies(sep=',').add_prefix(name+'_')
        df.drop([name],axis=1,inplace=True)
        dummies
        df = df.join(dummies)
        return df
    arr = list(data)
    for val in arr:
        if val == 'uid':
            continue
        data = dum(data,val)
    
    result = pd.merge(ratings, data, left_on='usera',right_on='uid')
    result.drop(['usera','uid'], axis=1, inplace=True)
    result = pd.merge(result, data, left_on='userb',right_on='uid')
    result.drop(['uid','userb'], axis=1, inplace=True)
    
    del data
    del ratings
    import gc
    gc.collect()
    return result

In [4]:
def train(data,length=743):
    def EmbeddingNetV3(n1_features,n2_features,n_latent_factors_user = 5,n_latent_factors_item = 8,n_users=743):
        model1_in = Input(shape=(n1_features,),name='userInput')
        model1_out = Embedding(input_dim = n_users+1, output_dim = n_latent_factors_user)(model1_in)
        model1_out = Flatten()(model1_out)
        model1_out = Dropout(0.2)(model1_out)

        model2_in = Input(shape=(n2_features,),name='itemInput')
        model2_out = Embedding(input_dim = n_users+1, output_dim = n_latent_factors_item)(model2_in)
        model2_out = Flatten()(model2_out)
        model2_out = Dropout(0.2)(model2_out)

        model = concatenate([model1_out, model2_out],axis=-1)
        model = LeakyReLU(alpha=0.15)(model)
        model = Dropout(0.2)(model)
        model = Dense(200)(model)
        model = LeakyReLU(alpha=0.15)(model)
        model = Dropout(0.2)(model)
        model = Dense(100)(model)
        model = LeakyReLU(alpha=0.15)(model)
        model = Dropout(0.2)(model)
        model = Dense(50)(model)
        model = LeakyReLU(alpha=0.15)(model)
        model = Dropout(0.2)(model)
        model = Dense(20)(model)
        model = LeakyReLU(alpha=0.15)(model)

        model = Dense(2, activation='softmax')(model)
        adam = Adam(lr=0.005)
        model = Model([model1_in, model2_in], model)
        model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
        return model
    
#     userb = pd.factorize(data.userb)[0]
    usera = data.iloc[:,1:525]
    userb = data.iloc[:,525:]
    n = length
    X = [usera,userb]
    y = data.rating
    
    encoder = LabelEncoder()
    encoder.fit(y)
    y = np_utils.to_categorical(encoder.transform(y), 2)
    
    callback = [EarlyStopping(patience=2,monitor='val_acc')]
    
    model = EmbeddingNetV3(n1_features=usera.shape[1],n2_features=userb.shape[1],n_users=n)
    history = model.fit(X,y,batch_size=1000,epochs=100,verbose=1,callbacks = callback)
    def saveModel(model):
        model_json = model.to_json()
        open('lounjee_rs_architecture.json', 'w').write(model_json)
        model.save_weights('lounjee_rs_weights.h5', overwrite=True)
    saveModel(model)
    return (model,history,X,y)

In [5]:
ratings = load_Data("mongodb://178.128.161.146:27017/")
data = pd.read_csv('./data1.csv')

In [6]:
result = data_Engineering(ratings,data,sampling=2)
train(result)
# (model,history,X,y) = train(result)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/100
544172/544172 [==============================] - 22s 41us/step - loss: 0.2944 - acc: 0.8728
Epoch 2/100
544172/544172 [==============================] - 23s 42us/step - loss: 0.1939 - acc: 0.9265
Epoch 3/100
544172/544172 [==============================] - 23s 42us/step - loss: 0.1665 - acc: 0.9391
Epoch 4/100
544172/544172 [==============================] - 23s 42us/step - loss: 0.1504 - acc: 0.9463
Epoch 5/100
544172/544172 [==============================] - 23s 42us/step - loss: 0.1397 - acc: 0.9509
Epoch 6/100
544172/544172 [==============================] - 23s 43us/step - loss: 0.1329 - acc: 0.9535
Epoch 7/100
544172/544172 [==============================] - 22s 41us/step - loss: 0.1268 - acc: 0.9559
Epoch 8/100
544172/544172 [==========================

544172/544172 [==============================] - 21s 38us/step - loss: 0.0745 - acc: 0.9756
Epoch 71/100
544172/544172 [==============================] - 21s 38us/step - loss: 0.0733 - acc: 0.9760
Epoch 72/100
544172/544172 [==============================] - 21s 38us/step - loss: 0.0724 - acc: 0.9765
Epoch 73/100
544172/544172 [==============================] - 21s 38us/step - loss: 0.0727 - acc: 0.9760
Epoch 74/100
544172/544172 [==============================] - 21s 38us/step - loss: 0.0732 - acc: 0.9764
Epoch 75/100
544172/544172 [==============================] - 21s 38us/step - loss: 0.0725 - acc: 0.9764
Epoch 76/100
544172/544172 [==============================] - 21s 39us/step - loss: 0.0723 - acc: 0.9763
Epoch 77/100
544172/544172 [==============================] - 21s 38us/step - loss: 0.0723 - acc: 0.9763
Epoch 78/100
544172/544172 [==============================] - 21s 38us/step - loss: 0.0716 - acc: 0.9764
Epoch 79/100
544172/544172 [==============================] - 21s 38

(<keras.engine.training.Model at 0x7f92104d1c88>,
 [array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 1, 0, ..., 1, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]), array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])],
 array([[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [8]:
from keras.models import model_from_json
def loadModel():
    model_architecture = 'lounjee_rs_architecture.json'
    model_weights = 'lounjee_rs_weights.h5'
    model = model_from_json(open(model_architecture).read())
    model.load_weights(model_weights)
    return model
model = loadModel()

In [13]:
usera = result.iloc[:,1:525]
userb = result.iloc[:,525:]
n = 743
X = [usera,userb]
y = result.rating
    
encoder = LabelEncoder()
encoder.fit(y)
y = np_utils.to_categorical(encoder.transform(y), 2)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.evaluate(X,y,verbose=1)

544172/544172 [==============================] - 22s 41us/step


[0.050272181908310974, 0.9827462640488669]

In [ ]:
# pred_y = model.predict(X)
# pred_y